# Three Dimensional

In [1]:
import gpytorch
import torch

import numpy as np
from numpy.random import multivariate_normal

import pandas as pd

import plotly
import plotly.graph_objects as go
import plotly.express as px


from scipy.stats import norm

import csv

In [2]:
x, y = torch.meshgrid(torch.linspace(0, 1, 10), torch.linspace(0, 1, 10), indexing="ij")
X = train_x = torch.cat((
    x.contiguous().view(x.numel(), 1),
    y.contiguous().view(y.numel(), 1)),
    dim=1
)
z = -(X[:, 1]*np.sin(X[:, 1] + X[:, 0]/2)-(X[:, 0])*np.sin(X[:, 0]+47))
print(X.size())
print(y.size())

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=2
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
        )

def gpytorchOptimize(xAxis, yPoints, kernel):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(xAxis, yPoints, likelihood, kernel)
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(10):
        optimizer.zero_grad()
        output = model(xAxis)
        loss = -mll(output, yPoints)
        loss.backward()
        optimizer.step()

    model.eval()
    observedPred = likelihood(model(xAxis))
    prediction = observedPred.mean.detach().numpy()
    

    return prediction

IndentationError: expected an indented block (Temp/ipykernel_22452/667040049.py, line 23)

In [ ]:
# kernel = gpytorch.kernels.MaternKernel(nu=0.5, ard_num_dims = 2)
# prediction = gpytorchOptimize(X, z, kernel)
x, y = torch.meshgrid(torch.linspace(0, 100, 20), torch.linspace(0, 100, 20), indexing="ij")
X = train_x = torch.cat((
    x.contiguous().view(x.numel(), 1),
    y.contiguous().view(y.numel(), 1)),
    dim=1
)
print(X)
def f(x1, x2):
    return (-x2+47)*np.sin(np.sqrt(x2+x1/2+47))-x1*np.sin(np.sqrt(x1-x2+47))
xResult = []
yResult = []
z =[]
for item in X:
    x = item[0].item()
    y = item[1].item()
    xResult.append(item[0].item())
    yResult.append(item[1].item())
    z.append(f(x, y))
    
# fig = go.Figure(data = go.Scatter3d(x = np.array(xResult), y = np.array(yResult), z = prediction, mode = "markers"))
# fig.show()

fig2 = go.Figure(data = go.Scatter3d(x = np.array(xResult), y = np.array(yResult), z = np.array(z), mode = "markers"))
fig2.show()

In [ ]:
i = -1.5
xData = []
yData = []
counter = 0

def f(x):
    return x**2+np.cos(x**2)+np.sin(x**3)

while (i <= 1.5):
    for x in range(8):
        xData.append(i)
    yData.append(f(i))
    yData.append(f(i+i/16))
    yData.append(f(i+i/18))
    yData.append(f(i+i/21))
    yData.append(f(i)*0.9)
    yData.append(f(i+i/16)*1.1)
    yData.append(f(i+i/18)*0.85)
    yData.append(f(i+i/20)*1.17)
    
    i+=0.1
    
xData = np.array(xData)
yData = np.array(yData)


for i in range(len(xData)):
    if (i >= len(xData)):
        break
    elif (i%3 == 0):
        xData = np.delete(xData, i)
        yData = np.delete(yData, i)
    elif (i%5 == 0):
        xData = np.delete(xData, [i, i+3])
        yData = np.delete(yData, [i, i+3])
    elif (i%5 == 0):
        xData = np.delete(xData, i)
        yData = np.delete(yData, i)

    
def remove(start, end, xData, yData):
    x = start
    array = []
    while (x < end):
        array.append(x)
        x+=1
    
    xData = np.delete(xData, array)
    yData = np.delete(yData, array)
    
    return xData, yData

xData, yData = remove(30, 35, xData, yData)
xData, yData = remove(50, 55, xData, yData)
xData, yData = remove(70, 80, xData, yData)
xData, yData = remove(100, 110, xData, yData)

print(len(xData))
fig = go.Figure(go.Scatter(x = xData, y = yData, mode = "markers"))
fig.show()

In [ ]:
def csvReader(filename):
    file = open(filename, 'r')
    rows = list(csv.reader(file, delimiter = ","))
    rows.pop(0)
    x1 = []
    x2 = []
    y = []
    for row in rows:
        x1.append(float(row[0]))
        x2.append(float(row[1]))
        y.append(float(row[2]))

    return torch.tensor(x1), torch.tensor(x2), torch.tensor(y)

x1, x2, y = csvReader("testdata/3dtest.csv")

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class Optimizer:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.numInput = self.X.size(0)
        self.params = len(self.X[0])
        print(self.params)
        if len(self.X != 0):
            if self.params == 1:
                self.domain = [self.X[0].item(), self.X[-1].item()]
                self.range = self.domain[1]-self.domain[0]
            elif self.params == 2:
                self.domain = [[self.X[0][0].item(), self.X[-1][0].item()], [self.X[0][1].item(), self.X[-1][1].item()]]
                self.range1 = self.domain[0][1]-self.domain[0][0]
                self.range2 = self.domain[1][1]-self.domain[1][0]
    def run(self):
        self.trainGP()
        for i in range(5):
            self.modelSurrogate()
            self.evaluateNext()
        return self.plot()

    def setOptimizationTarget(self, target):
        if target == "Maximize":
            self.target = max
            self.targetIndex = np.argmax
        else:
            self.target = min
            self.targetIndex = np.argmin
    
    def setKernel(self, kernelInput):
        self.kernel = gpytorch.kernels.MaternKernel(nu=0.5, ard_nums = self.params)
        if kernelInput == "matern":
            self.kernel = gpytorch.kernels.MaternKernel(nu=0.5, ard_nums = self.params) 
        if kernelInput == "sek":
            self.kernel = gpytorch.kernels.RBFKernel()  

    def setAcquisition(self, acquisitionInput):
        self.acquisition = self.probabilityOfImprovement 
        if acquisitionInput == "pi":
            self.acquisition = self.probabilityOfImprovement 
        
    def trainGP(self, trainIter = 10):
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()
        self.model = ExactGPModel(self.X, self.y, self.likelihood, self.kernel)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)

        self.model.train()
        self.likelihood.train()

        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.1)
        for i in range(trainIter):
            optimizer.zero_grad()
            output = self.model(self.X)
            loss = -mll(output, self.y)
            loss.backward()
            optimizer.step()

        self.model.eval()
        self.observedPred = (self.model(self.X))
    
    def modelSurrogate(self):
        self.prediction = self.likelihood(self.observedPred).mean.detach().numpy()

    def probabilityOfImprovement(self):
        if self.params == 1:
            exploreX = self.range*torch.rand(25)+self.domain[0]
            bestPoint = self.target(self.prediction)
            observedPred = (self.model(exploreX))
            exploreY = self.likelihood(observedPred).mean.detach().numpy()
            stdev = np.sqrt(observedPred.variance.detach().numpy())
            z = (exploreY - bestPoint)/stdev
            cdf = norm.cdf(z)
            if self.target == max:
                index = np.argmax(cdf)
            else:
                cdf = -cdf
                index = np.argmin(cdf)
                newX = exploreX[index]

            return newX.double(), torch.tensor([index])
        elif self.params == 2:
            exploreX1 = self.range1*torch.rand(int(5))+self.domain[0][0]
            exploreX2 = self.range2*torch.rand(int(5))+self.domain[1][0]
            exploreX = torch.cat((
                exploreX1.contiguous().view(exploreX1.numel(), 1),
                exploreX2.contiguous().view(exploreX2.numel(), 1)),
                dim=1
            )
            bestPoint = self.target(self.prediction)
            observedPred = (self.model(exploreX))
            exploreY = self.likelihood(observedPred).mean.detach().numpy()
            stdev = np.sqrt(observedPred.variance.detach().numpy())
            z = (exploreY - bestPoint)/stdev
            cdf = norm.cdf(z)
            if self.target == max:
                index = np.argmax(cdf)
            else:
                cdf = -cdf
                index = np.argmin(cdf)
            newX = exploreX[index]
            return newX, index
    def evaluateNext(self):
        if self.params == 1:
            newX, index = self.acquisition()
            self.X.index_add(0, index, newX)
        elif self.params == 2:
            newX, index = self.acquisition()
            print(newX.unsqueeze(0))
            self.X = torch.cat((self.X, newX.unsqueeze(0)), 0)
            print(self.X)
    def plot(self):
        if self.params == 1:
            figure = go.Figure(data = go.Scatter(x = self.X, y = self.prediction, name = "Prediction"))
            figure.add_trace(go.Scatter(x = self.X, y = self.y, name = "Training data", mode= "markers"))
            return figure, _
        elif self.params == 2:
            x1 = []
            x2 = []
            for item in self.X:
                x1.append(item[0].item())
                x2.append(item[1].item())
            figure = go.Figure(data = go.Scatter3d(x = x1, y = x2, z = self.prediction, mode = "markers"))
            figure.show()
            x, y = torch.meshgrid(torch.linspace(self.domain[0][0], self.domain[0][1], 10), torch.linspace(self.domain[1][0], self.domain[-1][1], 10), indexing="ij")
            X = torch.cat((
                x.contiguous().view(x.numel(), 1),
                y.contiguous().view(y.numel(), 1)),
                dim=1
            )
            prediction = self.likelihood(self.model(X)).mean.detach().numpy()
            prediction = prediction.reshape(10, 10)
            figure2 = go.Figure(data = go.Surface(z = prediction, x = x, y = y))
            figure2.show()
    def result(self):
        if self.params == 1:
            if self.target == max:
                index = self.prediction.argmax()
            else:
                index = self.prediction.argmin()
            bestX = self.X[index].item()
            bestY = self.prediction[index]
            bestX = round(bestX, 4)
            bestY = round(bestY, 4)
            if self.target == max:
                result = "Output maximized at param1 value of " + str(bestX) + " for a predicted output of " +  str(bestY)
            else:
                result = "Output minimized at param1 value of " + str(bestX) + " for a predicted output of " +  str(bestY)
            return result
        elif self.params == 2:
            if self.target == max:
                index = self.prediction.argmax()
            else:
                index = self.prediction.argmin()
            bestX1 = self.X[index][0].item()
            bestX2 = self.X[index][1].item()
            bestY = self.prediction[index]
            bestX1 = round(bestX1, 4)
            bestX2 = round(bestX2, 4)
            if self.target == max:
                result = "Output maximized at param1 value of " + str(bestX1) + " and param2 value of " + str(bestX2) + " for a predicted output of " +  str(bestY)
            else:
                result = "Output minimized at param1 value of " + str(bestX1) + " and param2 value of " + str(bestX2) + " for a predicted output of " +  str(bestY)
            return result
        else:
            if self.target == max:
                index = self.prediction.argmax()
            else:
                index = self.prediction.argmin()
            results = self.X[index]
            print(results)
            return results

In [ ]:
## Turn param1 and param2 inputs into torch form that GP can use
X = torch.cat((
    x1.contiguous().view(x1.numel(), 1),
    x2.contiguous().view(x2.numel(), 1)),
    dim=1
)

In [ ]:
optimizer = Optimizer(X, y)
optimizer.setOptimizationTarget("Maximize")
optimizer.setKernel("matern")
optimizer.setAcquisition("pi")
optimizer.trainGP()
optimizer.modelSurrogate()
optimizer.run()
optimizer.plot()
print(optimizer.result())

In [ ]:
X = optimizer.getX()
newX1 = torch.tensor[]

In [4]:
filename = 'testdata/multiparam.xlsx'
df = pd.read_excel(filename)
df = df.drop(index = 0)
A = torch.tensor(df["A"].to_numpy()[:23])
B = torch.tensor(df["B"].to_numpy()[:-4])
C = torch.tensor(df["C"].to_numpy()[:-4])
D = torch.tensor(df["d"].to_numpy()[:-4])
E = torch.tensor(df["E"].to_numpy()[:-4])
F = torch.tensor(df["F"].to_numpy()[:-4])
G = torch.tensor(df["G"].to_numpy()[:-4])
H = torch.tensor(df["H"].to_numpy()[:-4])
I = torch.tensor(df["I"].to_numpy()[:-4])
J = torch.tensor(df["J"].to_numpy()[:-4])
K = torch.tensor(df["K"].to_numpy()[:-4])
L = torch.tensor(df["L"].to_numpy()[:-4])
M = torch.tensor(df["M"].to_numpy()[:-4])
N = torch.tensor(df["N"].to_numpy()[:-4].astype('float64'))
O = torch.tensor(df["O"].to_numpy()[:-4])
P = torch.tensor(df["P"].to_numpy()[:-4])
Q = torch.tensor(df["Q"].to_numpy()[:-4])
R = torch.tensor(df["R"].to_numpy()[:-4])
S = torch.tensor(df["S"].to_numpy()[:-4])
T = torch.tensor(df["T"].to_numpy()[:-4])
U = torch.tensor(df["U"].to_numpy()[:-4])
V = torch.tensor(df["V"].to_numpy()[:-4])
W = torch.tensor(df["W"].to_numpy()[:-4])

In [5]:
X = A
y1 = torch.from_numpy(df["DT"].to_numpy()[:23].astype('float64'))
y2 = torch.from_numpy(df["r"].to_numpy()[:23].astype('float64'))
y3 = torch.from_numpy(df["R0"].to_numpy()[:23].astype('float64'))
y4 = torch.from_numpy(df["TL"].to_numpy()[:23].astype('float64'))
y = torch.stack([y1, y2, y3, y4], -1)

import math 

X = torch.stack([A], -1)
y1 = torch.from_numpy(df["DT"].to_numpy()[:23].astype('float64'))
y2 = torch.from_numpy(df["r"].to_numpy()[:23].astype('float64'))
y3 = torch.from_numpy(df["R0"].to_numpy()[:23].astype('float64'))
y4 = torch.from_numpy(df["TL"].to_numpy()[:23].astype('float64'))
y = torch.stack([y1, y2], -1)
print(X.dtype)
print(y.dtype)

torch.float64
torch.float64


In [8]:
X = A.float()
y1 = torch.from_numpy(df["DT"].to_numpy()[:23].astype('float64'))
y2 = torch.from_numpy(df["r"].to_numpy()[:23].astype('float64'))
y3 = torch.from_numpy(df["R0"].to_numpy()[:23].astype('float64'))
y4 = torch.from_numpy(df["TL"].to_numpy()[:23].astype('float64'))
y = torch.stack([y1, y2], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=2
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2)
model = MultitaskGPModel(X, y, likelihood)

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(10):
    optimizer.zero_grad()
    output = model(X)
    loss = -mll(output, y.float())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, 10, loss.item()))
    optimizer.step()

Iter 1/10 - Loss: 2.258
Iter 2/10 - Loss: 2.102
Iter 3/10 - Loss: 1.968
Iter 4/10 - Loss: 1.851
Iter 5/10 - Loss: 1.746
Iter 6/10 - Loss: 1.653
Iter 7/10 - Loss: 1.570
Iter 8/10 - Loss: 1.497
Iter 9/10 - Loss: 1.430
Iter 10/10 - Loss: 1.368
